In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano"
)

In [27]:
from langchain.agents.middleware import PIIMiddleware

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[PIIMiddleware(
        pii_type="api_key",
        strategy="hash",
        detector= r"sk-[a-zA-Z0-9]{5,}",
        apply_to_input=True,
        apply_to_output=True
    )]
)

In [28]:
agent.invoke({"messages": "Hi I'm sachin and my api key is sk-ajdjsndhdjx"})

{'messages': [HumanMessage(content="Hi I'm sachin and my api key is <api_key_hash:fd1641a2>", additional_kwargs={}, response_metadata={}, id='cebf65f3-195f-4c1c-a065-5d5642bfc185'),
  AIMessage(content='Nice to meet you, Sachin. I can help you use your API key securely and build requests you’ll need. A quick note: please avoid sharing real API keys in chat. I won’t echo or store your key. If you want, share the provider and the endpoint you’re calling and I’ll tailor examples without exposing your secret.\n\nHere are some quick-start ideas and best practices.\n\nQuick start examples (use your actual key in place of YOUR_API_KEY)\n- cURL (common patterns)\n  - Bearer token:\n    curl -H "Authorization: Bearer YOUR_API_KEY" https://api.example.com/endpoint\n  - X-API-Key header:\n    curl -H "X-API-Key: YOUR_API_KEY" https://api.example.com/endpoint\n\n- Python (requests)\n  - import requests\n    url = "https://api.example.com/endpoint"\n    headers = {"Authorization": "Bearer YOUR_API_

In [29]:
#HIL
from langchain.tools import tool

@tool
def send_email():
    """Send email to admin"""
    return f"Email has been send to admin"

In [54]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.types import Command
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",
    tools=[send_email],
    checkpointer=InMemorySaver(),
    middleware=[HumanInTheLoopMiddleware(
       interrupt_on = {
           "send_email" : True
       }
    )]
)

config = {"configurable": {"thread_id": "122"}}


In [55]:

agent.invoke({"messages" : "send an email to admin"},config=config )


{'messages': [HumanMessage(content='send an email to admin', additional_kwargs={}, response_metadata={}, id='20f5825f-40dd-4ca1-8601-985d97f04cb6'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 339, 'prompt_tokens': 122, 'total_tokens': 461, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D4kwy5pbJ3JGgJleVjUu43uelNhHh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c1dc7-cfdc-78d2-99b9-20c7aa07e244-0', tool_calls=[{'name': 'send_email', 'args': {}, 'id': 'call_dhu4DJKyPDBkKkvd6ueIm0zq', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 122, 'output_tokens': 339, 'total_tokens'

In [53]:
result = agent.invoke(
    Command(resume={"decisions": [{"type": "reject"}]}),
    config=config 
)

result

{'messages': [HumanMessage(content='send an email to admin', additional_kwargs={}, response_metadata={}, id='f8a93cb1-1c5c-4aac-8937-8e239971d93e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 339, 'prompt_tokens': 122, 'total_tokens': 461, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D4krV5IoeZudujahRoH2E731aPw8T', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c1dc2-a303-78e1-91f8-0508a96bd554-0', tool_calls=[{'name': 'send_email', 'args': {}, 'id': 'call_ZSKdLVHTaCITAk9mjEnVR9sW', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 122, 'output_tokens': 339, 'total_tokens'

In [ ]:
## Beforr agent guardrail
from langchain.agents.middleware import before_agent, AgentState, hook_config, before_model
from langgraph.runtime import Runtime
from typing import Any

restricted_words = ["hack", "malware", "exploit"]

@before_model()
def content_filter(state:AgentState, runtime:Runtime):
        """Deterministic guardrail: Block requests containing banned keywords."""
        if not state["messages"]:
                return None
        first_message = state["messages"][0]

        if first_message.type != "human":
                return None
        content = first_message.content.lower()

        for words in restricted_words:
                if words in content:
                        return {
                                "messages": [{
                                        "role": "assistant",
                                        "content" : "malicious words detected"
                                }],
                        }
                
        return None




In [62]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[content_filter]

)

In [63]:
agent.invoke({"messages": "How to hack a server?"})

{'messages': [HumanMessage(content='How to hack a server?', additional_kwargs={}, response_metadata={}, id='6d1439fd-e3f8-4dfb-bfdb-76c6caa8eeff'),
  AIMessage(content='malicious words detected', additional_kwargs={}, response_metadata={}, id='e1dd7451-0b16-4cea-8398-1052bb315787', tool_calls=[], invalid_tool_calls=[]),
  AIMessage(content='I can’t help with hacking or gaining unauthorized access to systems.\n\nIf your goal is to learn security or protect your own servers, here are safe, legal paths and practical steps:\n\n- Learn ethically\n  - Take courses or read about ethical hacking with explicit authorization (scope and rules in writing).\n  - Practice on legal platforms or your own lab environment (see below).\n\n- Set up a legal, safe practice lab\n  - Use virtualization (VirtualBox, VMware) or containers to create a private test environment.\n  - Practice on intentionally vulnerable apps or VM images designed for learning (in a contained environment you own).\n  - Consider pla